Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

In [ ]:
# Enzyme-constrained Yeast 8
model = cobra.io.load_yaml_model("./models/ecYeastGEMfull.yml")
model

# Inspect functions, metabolites, and reactions

Objective function is `GROWTH` in the yeast-GEM model, inspecting it and related reactions:

In [ ]:
model.reactions.get_by_id('r_4041')

In [ ]:
model.metabolites.get_by_id('s_0450')

In [ ]:
# This is the biomass pseudoreaction -- we're interested in the stoichiometry of this.
biomass = model.reactions.get_by_id('r_4041')
biomass
# Conveniently, it has one 'metabolite' for all lipids, one for all proteins,
# one for RNA, etc.  We want to remove each in turn, so if it's written in this way,
# it's super easy to do so.

In [ ]:
def print_formula_weights(reaction):
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id}: MW {reactant.formula_weight}')
    print('products')
    for product in reaction.products:
        print(f'{product.id}: MW {product.formula_weight}')
        
print_formula_weights(biomass)

Inspecting each type of macromolecule...

## Lipid

In [ ]:
model.metabolites.get_by_id('s_1096')

In [ ]:
# Lipid pseudoreaction is combining 'bulk' lipid backbone and 'bulk' lipid chain substrates
lipid_reaction = model.reactions.get_by_id('r_2108')
lipid_reaction

In [ ]:
print_formula_weights(lipid_reaction)

In [ ]:
model.metabolites.get_by_id('s_3746')

In [ ]:
lipid_backbone_reaction = model.reactions.get_by_id('r_4063')
lipid_backbone_reaction

In [ ]:
print_formula_weights(lipid_backbone_reaction)

In [ ]:
model.metabolites.get_by_id('s_0694')

In [ ]:
fatty_acid_backbone_reaction = model.reactions.get_by_id('r_3978')
fatty_acid_backbone_reaction

Protein

In [ ]:
model.metabolites.get_by_id('s_3717')

In [ ]:
# Protein pseudoreaction is combining aminoacyl-tRNAs
model.reactions.get_by_id('r_4047')

Carbohydrates

In [ ]:
model.metabolites.get_by_id('s_3718')

In [ ]:
# Carbohydrate pseudoreaction is all the storage and structural (cell wall) ones together
model.reactions.get_by_id('r_4048')

DNA

In [ ]:
model.metabolites.get_by_id('s_3720')

In [ ]:
# DNA pseudoreaction is combining dNTPs
model.reactions.get_by_id('r_4050')

RNA

In [ ]:
model.metabolites.get_by_id('s_3719')

In [ ]:
# RNA pseudoreaction is combining NTPs
model.reactions.get_by_id('r_4049')

# Unmodified model

Limit glucose uptake (minimal media)

In [ ]:
# TODO, find a sensible number.
# Sánchez et al. (2017) keep the uptake for the carbon substrate (glucose, sucrose, etc.) unrestricted.
# Alternatively, I can use the same logic as for the citramalate project.
model.reactions.EX_glc__D_e.bounds = (-18.5, 1)

Simulate model and draw fluxes through central carbon metabolism

In [ ]:
solution = model.optimize()
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b

Get flux through biomass reaction

In [ ]:
biomass = model.reactions.get_by_id('r_4041')
print(f'Flux through biomass reaction is {biomass.flux:.4f} mmol/(gDW h)')

Estimate timescale for biomass synthesis

In [ ]:
# Define constants
CELL_DRY_MASS = 15e-12 # g
# Using C:H(1.613):O(0.557):N(0.158) from https://bionumbers.hms.harvard.edu/bionumber.aspx?id=101801
# MOLWEIGHT_BIOMASS = 12.011 + 1.613*1.00784 + 0.557*15.999 + 0.158*14.0067
# MOLWEIGHT_BIOMASS = 0.966 # g/mmol, Takhaveev et al. (2023)

In [ ]:
MOLWEIGHT_BIOMASS

In [ ]:
biomass_time = 1/(biomass.flux * MOLWEIGHT_BIOMASS)
print(f'Estimated time: {biomass_time:.4f} hours')

## Modify biomass reaction by ablating each type of macromolecule

In [ ]:
CELL_DRY_MASS = 15e-12 # g

class BiomassComponent():
    def __init__(
        self,
        metabolite_label,
        metabolite_id,
        pseudoreaction,
        molecular_mass,
        mass_per_cell,
        copy_number,
    ):
        self.metabolite_label = metabolite_label
        self.metabolite_id = metabolite_id
        self.pseudoreaction = pseudoreaction
        self.molecular_mass = molecular_mass # g/mmol
        self.mass_per_cell = mass_per_cell # g
        self.copy_number = copy_number
        
        self.ablated_flux = None # mmol/(g DW . h)
        self.est_time = None # h
        
    def get_est_time(self):
        #self.est_time = self.mass_per_cell/(CELL_DRY_MASS * self.ablated_flux * self.copy_number * self.molecular_mass)
        self.est_time = self.mass_per_cell/(CELL_DRY_MASS * self.ablated_flux * self.molecular_mass)
        

In [ ]:
model_saved = cobra.io.load_yaml_model("./models/ecYeastGEMfull.yml")

In [ ]:
# TODO:
# - Create CSV table containing these
# - Create a class builder that builds these classes based on the CSV table
# - FURTHER: make it able to deal with ranges of values (lower limit, upper limit)

Lipids = BiomassComponent(
    metabolite_label='lipid',
    metabolite_id='s_1096',
    pseudoreaction='r_2108',
    molecular_mass=0.800,
    mass_per_cell=900e-15,
    copy_number=1e9,
)

Proteins = BiomassComponent(
    metabolite_label='protein',
    metabolite_id='s_3717',
    pseudoreaction='r_4047',
    molecular_mass=55,
    mass_per_cell=7650e-15,
    copy_number=1e8,
)

Carbohydrates = BiomassComponent(
    metabolite_label='carbohydrate',
    metabolite_id='s_3718',
    pseudoreaction='r_4048',
    molecular_mass=1000, # 'variable', so taking the E. coli one
    mass_per_cell=(75+3450)*1e-15, # 'storage carbohydrates' + 'structural polymers'
    copy_number=2122804981, # estimated from above & avogadro's const
)

DNA = BiomassComponent(
    metabolite_label='DNA',
    metabolite_id='s_3720',
    pseudoreaction='r_4050',
    molecular_mass=2.5e5,
    mass_per_cell=75e-15,
    copy_number=16,
)

RNA = BiomassComponent(
    metabolite_label='RNA',
    metabolite_id='s_3719',
    pseudoreaction='r_4049',
    molecular_mass=1e2, # 1e4 -- 1e6 Da
    mass_per_cell=1650e-15,
    copy_number=4e6,
)

In [ ]:
biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

all_pseudoreaction_ids = [
    (biomass_component.metabolite_label, biomass_component.pseudoreaction)
    for biomass_component in biomass_component_list
]
all_pseudoreaction_ids.append(('objective','r_4041'))

# original
model = model_saved.copy()
biomass_reaction = model.reactions.get_by_id('r_4041')
biomass_reaction.bounds = (0, 1000)
solution = model.optimize()
print('original')
print(biomass_reaction)
print(f'flux of objective function: {biomass_reaction.flux:.4f} mmol/(gDW h)')
for metabolite_label, pseudoreaction_id in all_pseudoreaction_ids:
    print(f'{metabolite_label} pseudoreaction ({pseudoreaction_id}) flux = {model.reactions.get_by_id(pseudoreaction_id).flux}')
plt.subplots()
plt.bar(
    [label for (label, _) in all_pseudoreaction_ids],
    [model.reactions.get_by_id(pseudoreaction_id).flux for (_, pseudoreaction_id) in all_pseudoreaction_ids],
)
plt.ylim((0,0.15))
plt.title('original')
#print(f'estimated time: {biomass_component.est_time:.4f} hours')
print('\n')

# ablated
for biomass_component in biomass_component_list:
    # get model
    model = model_saved.copy()
    # model.reactions.EX_glc__D_e.bounds = (-18.5, 1) # Limit glucose uptake.  TODO, find a sensible number
    biomass_reaction = model.reactions.get_by_id('r_4041')
    biomass_reaction.bounds = (0, 1000)
    
    # boilerplate: lookup
    to_ablate = all_metabolite_ids.copy()
    to_ablate.remove(biomass_component.metabolite_id)
    to_ablate_keys = [
        model.metabolites.get_by_id(metabolite_id)
        for metabolite_id in to_ablate
    ]
    to_ablate_dict = dict(zip(to_ablate_keys, [-1]*len(to_ablate_keys)))
    
    # ablate metabolites from biomass reaction
    biomass_reaction.subtract_metabolites(to_ablate_dict)
    
    # optimise model
    solution = model.optimize()
    biomass_component.ablated_flux = biomass_reaction.flux
    biomass_component.get_est_time()
    print(f'prioritising {biomass_component.metabolite_label}')
    print(biomass_reaction)
    print(f'flux of ablated objective function: {biomass_component.ablated_flux:.4f} mmol/(gDW h)')
    for metabolite_label, pseudoreaction_id in all_pseudoreaction_ids:
        print(f'{metabolite_label} pseudoreaction ({pseudoreaction_id}) flux = {model.reactions.get_by_id(pseudoreaction_id).flux}')
    plt.subplots()
    plt.bar(
        [label for (label, _) in all_pseudoreaction_ids],
        [model.reactions.get_by_id(pseudoreaction_id).flux for (_, pseudoreaction_id) in all_pseudoreaction_ids],
    )
    plt.ylim((0,0.15))
    plt.title(f'prioritising {biomass_component.metabolite_label}')
    #print(f'estimated time: {biomass_component.est_time:.4f} hours')
    print('\n')
    
total_time = sum([biomass_component.est_time for biomass_component in biomass_component_list])
#print(f'sum of times: {total_time:.4f} hours')

Draw fluxes through central carbon metabolism

In [ ]:
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b

Misc

In [ ]:
solution.to_frame()[solution.fluxes > 0]